In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env2 import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
import agents
import sender
from agents import find_lengths

In [2]:
NUM_OPTIONS = 10
NUM_DISTRACTORS = 9
BATCH_SIZE = 32
HIDDEN_SIZE = 44
EMBED_DIM = 44
VOCAB_SIZE = 99
MAX_LEN = 10
NUM_EPOCHS = 1
TRAINING = True
sender_entropy_coeff = 0.015 #wie bei Ossenkopf
receiver_entropy_coeff = 0.0 # wie bei Ossenkopf


In [3]:

vocabulary = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.xml"
    file = minidom.parse(os.path.join(os.path.join('visa_dataset', 'UK'), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        vocabulary.addConcept(concept)


addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTEFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

for concept in vocabulary.concept_list:
    vocabulary.parseConcept(concept)

In [4]:
#Initialisiere die Agents
speakerEncoder = sender.Sender(num_options=NUM_OPTIONS, batch_size=BATCH_SIZE)
speakerEncoderOnlyTarget = sender.SenderOnlyTarget(batch_size=BATCH_SIZE)
speakerDecoder = sender.Sender_LSTM(embed_dim=EMBED_DIM,
                                    num_cells=1,
                                    hidden_size=1, 
                                    max_len=MAX_LEN)
receiverEncoder = agents.Receiver(num_options = NUM_OPTIONS)
receiverDecoder = agents.Receiver_LSTM(agent=receiverEncoder, 
                                       vocab_size=VOCAB_SIZE,
                                       embed_dim=EMBED_DIM, 
                                       hidden_size=HIDDEN_SIZE)
guesser = agents.AuxillaryNetwork(hidden_size=HIDDEN_SIZE)

In [5]:
#the losses

#loss of guessing the correct target
def loss(input_concepts, receiver_output, targets):
    """
    receiver_output ist was von receiver_sampling zurückgegeben wird
    LABELS PRINTEN IN OSSSENKOPF NOTEBOOK
    """
    guesses = []
    
    for i in range(len(receiver_output)):
        guesses.append(input_concepts[i][receiver_output[i]])
            
    guesses = tf.constant(guesses)
    targets = tf.constant(targets)
    acc = np.sum(guesses == targets) / guesses.shape[0]
    
    return -acc

#auxiliary loss to promote empathy
def auxiliary_loss(receiver_thoughts, 
                  # _message, _receiver_input, 
                   guesser_output, 
                   #_labels,
                   weight=0.2):
    mae = tf.keras.losses.MeanAbsoluteError(reduction = 'none')
    loss = mae(receiver_thoughts, guesser_output)
    loss *= weight
    
    return loss

In [6]:
optim_guesser = tf.keras.optimizers.Adam(learning_rate = 1e-2)
optim_receiver = tf.keras.optimizers.Adam(learning_rate = 1e-2)
optim_sender = tf.keras.optimizers.Adam(learning_rate = 1e-3)

In [7]:
for i in range(NUM_EPOCHS):
    # 1. data holen, labels?
    ####Fehlt hier noch
    data = ConceptData(voc=vocabulary, num_distractors=NUM_DISTRACTORS, batch_size=BATCH_SIZE)
    input_concepts, sender_input, targets, receiver_input = data.getInput()
    
    
    with tf.GradientTape() as tape:
    
        ##2. durch agents laufen lassen
        output_sender = speakerEncoder(sender_input)
        message, log_prob_s, entropy_s , prev_hidden = speakerDecoder(output_sender)

        #einer- Dimension rausbekommen
        receiver_input = tf.squeeze(tf.convert_to_tensor(receiver_input))
        #richtige Reihenfolge der Dimensionen
        receiver_input2 = tf.transpose(receiver_input, [1,0,2])
        receiver_input2 = receiver_input2.numpy()

        sample, log_prob_r, entropy_r, last_hidden = receiverDecoder(message=message, 
                                                                     batch_size=BATCH_SIZE, 
                                                                     max_len=MAX_LEN,
                                                                     receiver_input=receiver_input2)
        guesser_output = guesser(prev_hidden)

        output_receiver = []
        for j in range(BATCH_SIZE):
            output_receiver.append(input_concepts[i][sample[i]])


        # 3. Loss berechnen
        auxLoss = auxiliary_loss(last_hidden, guesser_output)
        # labels fehlen noch
        loss = loss(input_concepts=input_concepts, 
                    receiver_output=sample, 
                    targets=targets)

        # compute effective entropy and log_prob of output before and including eos
        message_lengths = find_lengths(message)

        effective_entropy_s = tf.zeros_like(entropy_r)
        effective_entropy_s = np.array(effective_entropy_s)
        effective_log_prob_s = tf.zeros_like(log_prob_r)
        effective_log_prob_s = np.array(effective_log_prob_s)

        #print(message.shape)

        for k in range(message.shape[0]):
            for l in range(message.shape[1]):
                not_eosed = float(k < message_lengths[k])
                effective_entropy_s[k] += entropy_s[k, l] * not_eosed
                effective_log_prob_s[k] += log_prob_s[k, l] * not_eosed

        effective_entropy_s = tf.convert_to_tensor(effective_entropy_s)
        effective_log_prob_s = tf.convert_to_tensor(effective_log_prob_s)

        weighted_entropy = entropy_r * receiver_entropy_coeff + sender_entropy_coeff * effective_entropy_s
        log_prob = log_prob_r + effective_log_prob_s
        loss = tf.math.reduce_mean(loss * log_prob) - weighted_entropy

        #optimization
        receiver_gradients = tape.gradient(loss, receiverDecoder.trainable_variables)
        guesser_gradients = tape.gradient(auxLoss, guesser.trainable_variables)
        
    
    with tf.GradientTape() as tape:
        # muss der eine Receiver im Konstruktor vom anderem erstellt werden damit das funktioniert?
        receiver_gradients = tape.gradient(loss, receiverDecoder.trainable_variables)
        #print(receiver_gradients)
    optim_receiver.apply_gradients(zip(receiver_gradients, receiverDecoder.trainable_variables))
    
    with tf.GradientTape() as tape:
        guesser_gradients = tape.gradient(auxLoss, guesser.trainable_variables)
        #print(guesser_gradients)
    optim_guesser.apply_gradients(zip(guesser_gradients, guesser.trainable_variables))
    
    with tf.GradientTape() as tape:
        senderLSTM_gradients = tape.gradient(loss, speakerDecoder.trainable)
        senderEncoder_gradients = guesser_gradients + senderLSTM_gradients
    optim_sender.apply_gradients(zip(senderLSTM_gradients, speakerDecoder.trainable_variables))
    optim_sender.apply_gradients(zip(senderEncoder_gradients, senderEncoder.trainable_variables))
        
    
    
    
    #testing <-> trainifehlt noch
    

shape sender input:
(32, 10, 1, 595)
32
input-shape:
(32, 10, 44)
10


ValueError: No gradients provided for any variable: ['receiver_lstm/receiver/encoder/dense_3/kernel:0', 'receiver_lstm/receiver/encoder/dense_3/bias:0', 'receiver_lstm/embedding/embeddings:0', 'receiver_lstm/lstm_1/lstm_cell_1/kernel:0', 'receiver_lstm/lstm_1/lstm_cell_1/recurrent_kernel:0', 'receiver_lstm/lstm_1/lstm_cell_1/bias:0'].